# Internal Functions

When debugging issues or testing configurations, it may be advantagous to run Isofit in a more controlled manner than the end-to-end Apply OE pipeline. This notebook gives a quick introduction to the structure of Isofit and how it can be used more like a Python package.

NOTE: Isofit is actively developed, so while we make sure that Apply OE is stable from a CLI standpoint, individual functions are subject to change.

We'll start with the medium example found at `~/.isofit/imagecube/medium`. First, run Apply OE on this example, or for more adept users, run Apply OE with the `--config_only` flag. The objective here, is to generate an Isofit config file at `~/.isofit/imagecube/small/config/ang20170323t202244_isofit.json`. This saves us the trouble of building the configuration file manually. We can edit the Apply OE generated configuration as needed.

In [23]:
# Imports
from pathlib import Path

from matplotlib import pyplot as plt
import numpy as np
from spectral import envi

from isofit.core.common import envi_header
from isofit.core.isofit import Isofit
from isofit.configs import configs
from isofit.core.forward import ForwardModel
from isofit.inversion.inverse import Inversion
from isofit.core.geometry import Geometry
from isofit.core.fileio import IO
from isofit.inversion.inverse_simple import (
    invert_algebraic,
    invert_analytical,
    invert_simple,
)

In [7]:
# Load the input data
rdn_path = Path('~/.isofit/imagecube/medium/ang20170323t202244_rdn_7k-8k')
loc_path = Path('~/.isofit/imagecube/medium/ang20170323t202244_loc_7k-8k')
obs_path = Path('~/.isofit/imagecube/medium/ang20170323t202244_obs_7k-8k')

rdn = envi.open(envi_header(str(rdn_path.expanduser())))
loc = envi.open(envi_header(str(loc_path.expanduser())))
obs = envi.open(envi_header(str(obs_path.expanduser())))

rdn_im = rdn.open_memmap(interleave='bip')
loc_im = loc.open_memmap(interleave='bip')
obs_im = obs.open_memmap(interleave='bip')

Much of the internal Isofit structure is object-based.

The `config` object contains the full Isofit config. This is constructed directly from the config `.json` file:

In [22]:
# Initialize the config object
config_file = Path('~/.isofit/imagecube/small/config/ang20170323t202244_isofit.json')
config =  configs.create_new_config(str(config_file.expanduser()))

print('The config object:')
print(config)
print()

print('Config attributes:')
[print(key) for key in vars(config).keys()]
print()

print('Example:')
print(f'Forward model type: {config._forward_model_type}')
print(f'Forward model: {config.forward_model}')

The config object:

Config attributes:
_input_type
input
_output_type
output
_forward_model_type
forward_model
_implementation_type
implementation

Example:
Forward model type: <class 'isofit.configs.sections.forward_model_config.ForwardModelConfig'>
Forward model: <isofit.configs.sections.forward_model_config.ForwardModelConfig object at 0x109ed3d90>


In [ ]:
multi_fm = ForwardModel(config)
multi_iv = Inversion(config, multi_fm)